# <font color="#003660">Applied Machine Learning for Text Analysis (M.184.5331)</font>
  

# <font color="#003660">Week 2: Unsupervised NLP</font>

# <font color="#003660">Notebook 1: Explore Pre-trained Word Embeddings</font>

<center><br><img width=256 src="https://raw.githubusercontent.com/olivermueller/aml4ta-2021/main/resources/dag.png"/><br></center>

<p>
<center>
<div>
    <font color="#085986"><b>By the end of this lesson, you ...</b><br><br>
        ... understand what word embeddings are, <br>
        ... are able to to load and use pre-trained word embeddings for determining semantic similarity between words, <br>
        ... are able to visualize word embeddings, and <br>
        ... can answer word analogy queries with word embeddings.
    </font>
</div>
</center>
</p>

# Import packages

As always, we first need to load a number of required Python packages:
- `pandas` provides high-performance, easy-to-use data structures and data analysis tools.
- `numpy` is a library adding support for large, multi-dimensional arrays and matrices, along with a large collection of high-level mathematical functions to operate on these arrays.
- `spacy` offers industrial-strength natural language processing.
- `gensim` is a fast library for training of vector embeddings and topic models.
- `sklearn` is the de-facto standard machine learning package in Python.
- `plotly` is a library for creating interactive plots.

In [ ]:
import pandas as pd
import numpy as np
import pickle
import spacy
from gensim.models import word2vec
from gensim.models import KeyedVectors
import gensim.downloader as api
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import plotly.express as px
import warnings
warnings.filterwarnings("ignore")

# What are word embeddings?

Word embeddings (e.g., word2vec, Glove) are an alternative to representing words through one-hot encoding. In contrast to one-hot encoding, which are hard-coded high-dimensional and sparse representations, word embeddings are lower-dimensional dense representations that are learned from the data. Word embeddings represent each word in a dictionary by a real-valued numeric vector. (Chollet, 2018)

<center><img width=512 src="https://git.uni-paderborn.de/data.analytics.teaching/aml4ta-2020/-/raw/master/week_3/images/one-hot_vs_word-embeddings.png"/></center>

In addition, word embeddings are able to capture the semantic meaning of words and map it into geometric space. This is achieved by assigning a numeric vector to each word in the vocabulary, such that the distance (e.g., cosine distance) between any two word vectors would capture part of the semantic relationship between the two associated words. For example, "apple" and "dog" are words that are semantically quite different, so a reasonable embedding space would represent them as vectors that would be very far apart. But "kitchen" and "fridge" are related words, so their vectors hould be close to each other. (Chollet, 2018)

Ideally, in a good embeddings space, the path (which is a vector itself) to go from "kitchen" to "fridge" would capture precisely the semantic relationship between these two concepts. This idea is illustrated in the following figure (adapted from Chollet, 2018).

<center><img width=768 src="https://git.uni-paderborn.de/data.analytics.teaching/aml4ta-2020/-/raw/master/week_3/images/wolf_dog_tiger_cat.png"/></center>

# Download pre-trained word embeddings

Download pre-trained word vectors from Gensim-data. The word vectors have 300 dimensions and were learned from 6 billion tokens from Wikipedia (2014) and Gigaword 5 (https://catalog.ldc.upenn.edu/LDC2011T07). See https://radimrehurek.com/gensim/models/keyedvectors.html#module-gensim.models.keyedvectors for documentation.

In [ ]:
word_vectors = api.load("glove-wiki-gigaword-300")

# Explore embeddings

Look at a single vector.

In [ ]:
word_vectors["man"]

In [ ]:
len(word_vectors["man"])

Use Gensim's built-in function most_similar() to retrieve most similar words to a given word.

In [ ]:
word_vectors.most_similar("man")

In [ ]:
word_vectors.most_similar("woman")

Which word doesn't belong to the set?

In [ ]:
word_vectors.doesnt_match(["red", "green", "blue", "sky"])

Let's look at some analogies using vector arithmetic: King – Man + Woman = ?

In [ ]:
word_vectors.most_similar(positive=['king', 'woman'], negative=['man'])

Berlin – Germany + France = ?

In [ ]:
word_vectors.most_similar(positive=['berlin', 'france'], negative=['germany'])

Wolf - Dog + Cat = ?

In [ ]:
word_vectors.most_similar(positive=['wolf', 'cat'], negative=['dog'])

Now it's your turn. Explore the similarities and analogies between word embeddings...

In [ ]:
# YOUR CODE GOES HERE


# Visualize embeddings

Get a list of all the words in the vocabulary.

In [ ]:
vocab = list(word_vectors.key_to_index)

Retrieve the associated word embedding vectors from the model.

In [ ]:
X = word_vectors[vocab]

Reduce the dimensionality of the data with PCA.

In [ ]:
X_pca = PCA(n_components=2).fit_transform(X)

Reformat data, add similarity to a "seed" word, and create an interactive scatterplot with plotly.

In [ ]:
pca_df = pd.DataFrame(X_pca, index=vocab, columns=['x', 'y'])
pca_df["word"] = vocab

seed = "berlin"
pca_df["sim"] = 0

for word, sim in word_vectors.most_similar(seed, topn=100):
    pca_df.loc[word, 'sim'] = sim

# filter to 100 most similar words?
# pca_df = pca_df[pca_df["sim"]>0]

fig = px.scatter(pca_df, x="x", y="y", color="sim",
                 hover_data=["word"],
                 range_x = [-6, 6], range_y = [-4, 4],
                 opacity = 0.2, color_continuous_scale='agsunset_r')
fig.show()